## Tips and Hints
It is often easier to debug the code using the debugger and run the code in python instead of through the Jupyter file. From a new terminal, activate your virtual environment and run the unit test file (below assumes windows for activating the virtual environment):

```
.\mav_venv\Scripts\activate
python .\mav_sim\unit_tests\ch2_transforms_tests.py
```

To run the code within the mav sim from the terminal, you will activate your environment and run the book assignment for chapter 2
```
.\mav_venv\Scripts\activate
python .\book_assignments\mavsim_chap2.py
```

## Problem 1: Basic concentric frame transforms
Most of the frames have the same origin. The code in chap2/transforms.py calculates the rotation matrices used to transform between these frames. Correctly implement the following functions:
* rot_x: calculate elementary rotation matrix about x-axis
* rot_y: calculate elementary rotation matrix about y-axis
* rot_z: calculate elementary rotation matrix about z-axis
* rot_v_to_v1: calculates the rotation from frame v to v1
* rot_v1_to_v2: calculates the rotation from frame v1 to v2
* rot_v2_to_b: calculates the rotation from v2 to body frame
* rot_b_to_s: calculates the rotation from body to stability frame
* rot_s_to_w: calculates the rotation from stability to wind frame

*Hint:* You should only compute the cosine and sine of the angle in *rot_x*, *rot_y*, and *rot_z*. All the remaining functions should call those functions (i.e., one line change from what they currently are)

Use these function to compute the following. Assume that $\psi = \frac{\pi}{4}$, $\theta = 0.3$, $\phi = 0.25$, $\alpha = 0.1$, and $\beta = 0.15$. Display the results in the exported pdf.
* Compute $p_1^{v1}$ given $p_1^v = \begin{bmatrix} 1 \\ 2 \\ 3 \end{bmatrix}$
* Compute $p_1^{w}$
* Compute $p_2^{s}$ given $p_2^{v2} = \begin{bmatrix} 4 \\ 5 \\ 6 \end{bmatrix}$
* Compute $p_2^v$



In [3]:
import numpy as np # Imports the numpy library and creates the alias np
from IPython.display import display # Used to display variables nicely in Jupyter
from mav_sim.chap2.transforms import rot_v_to_v1, rot_v1_to_v2, rot_v2_to_b, rot_b_to_s, rot_s_to_w, rot_x, rot_y, rot_z

# Calculate the required rotation matrices
psi = np.pi/4
theta = 0.3
phi = 0.25
alpha = 0.1
beta = 0.15

# Initialize p1 and p2
p_1_v = np.array([[1],[2],[3]])
p_2_v2 = np.array([[4],[5],[6]])

# Calculate p_1^v1
p_1_v1 = rot_v_to_v1(psi)@p_1_v   
print('p_1^v1 = ')
display(p_1_v1)

# Calculate p_1^w
p_1_w = rot_s_to_w(beta)@rot_b_to_s(alpha)@rot_v2_to_b(phi)@rot_v1_to_v2(theta)@p_1_v1
print('p_1^w = ')
display(p_1_w)

# Calculate p_2^s
p_2_s = rot_b_to_s(alpha)@rot_v2_to_b(phi)@p_2_v2
print('p_2^s = ')
display(p_2_s)

# Calculate p_2^v
p_2_v = np.linalg.inv(rot_v_to_v1(psi))@np.linalg.inv(rot_v1_to_v2(theta))@p_2_v2
print('p_2^v = ')
display(p_2_v)

#calculate R_x(pi/3)@R_z(pi/4)
R = rot_x(np.pi/3)@rot_z(np.pi/4)
print('R = ')
display(R)

#caclultae R_x(pi/6)@inv(R_y(pi/3))@R_z(3pi/4)
R = rot_x(np.pi/6)@np.linalg.inv(rot_y(np.pi/3))@rot_z(3*np.pi/4)
print('R = ')
display(R)


p_1^v1 = 


matrix([[2.12132034],
        [0.70710678],
        [3.        ]])

p_1^w = 


matrix([[1.66990864],
        [1.31449478],
        [3.07953058]])

p_2^s = 


...

matrix([[0.42035179],
        [7.4914196 ],
        [4.54993811]])

R = 


matrix([[ 0.70710678,  0.70710678,  0.        ],
        [-0.35355339,  0.35355339,  0.8660254 ],
        [ 0.61237244, -0.61237244,  0.5       ]])

R = 


matrix([[-0.35355339,  0.35355339,  0.8660254 ],
        [-0.30618622, -0.91855865,  0.25      ],
        [ 0.88388348, -0.1767767 ,  0.4330127 ]])




## Problem 2: Compound rotation matrices
The transform from the vehicle frame to the body frame can be written as a compound of three rotation matrices (and so can the inverse transform). However, these matrices are used so often that it is nice to avoid multiplying these three matrices each time the transform is needed. 

Implement the following functions:

* rot_v_to_b: calculates the rotation from vehicle to body frame
* rot_b_to_v: calculates the rotation from body frame to vehicle frame

*Hint:* You really only need to implement one of them and then use a transpose for the other

Using the same values as above, show that your implementation produces the same rotation matrices as three elementary matrices multiplied together. Display the difference in the exported pdf.


In [3]:
from mav_sim.chap2.transforms import rot_v_to_b, rot_b_to_v

# Calculate the rotation matrices as compound rotation matrices (i.e., matrix multiplication)
rot_v_b_1 = rot_v2_to_b(phi)@rot_v1_to_v2(theta)@rot_v_to_v1(psi)
rot_b_v_1 = np.linalg.inv(rot_v_b_1)

# Calculate the rotation matrices using the functions
rot_v_b_2 = rot_v_to_b(psi, theta, phi)
rot_b_v_2 = np.linalg.inv(rot_v_b_2)

# Calculate and display the difference
R_v_to_b_diff = rot_v_b_1 - rot_v_b_2
print("R_v_to_b_diff")
display(R_v_to_b_diff)

R_b_to_v_diff = rot_b_v_1 - rot_b_v_2
print("R_b_to_v_diff")
display(R_b_to_v_diff)

R_v_to_b_diff


matrix([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

R_b_to_v_diff


matrix([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

## Problem 3: Tranform to vehicle frame
Converting to and from the inertial frame requires translation. Implement the following functions:
* trans_i_to_v: transforms a point from inertial frame to the vehicle frame
* trans_v_to_i: transforms a point from vehicle frame to the inertial frame
* trans_i_to_b: transforms a point from inertial frame to the body frame
* trans_b_to_i: transforms a point from the body frame to the inertial frame

Note that the transform between inertial and the vehicle frame is purely translational. The transform between the vehicle and body frame is purely rotational. Thus, you can use the functions already implemented to make the *trans_i_to_b* and *trans_b_to_i* functions quite simple.

Given that the UAV is in the position $p_n = 1$, $p_e = 2$, and $p_d = 3$ with the angles defined as before, transform the following points to the body frame using the implemented functions:

$p_3^i = \begin{bmatrix} 1 \\ 2 \\ 3\end{bmatrix}$

$p_4^i = \begin{bmatrix} 3 \\ 2 \\ 1\end{bmatrix}$

Transform the following point in the body frame to the inertial frame

$p_5^b = \begin{bmatrix} 1 \\ 2 \\ 3\end{bmatrix}$

Transform the following point in the wind frame to the inertial frame

$p_6^w = \begin{bmatrix} 1 \\ 2 \\ 3\end{bmatrix}$

Display the results in the exported pdf.

In [6]:
from mav_sim.chap2.transforms import trans_i_to_b, trans_b_to_i

# Create the pose of the aircraft
class Pose:
    def __init__(self) -> None:
        self.north: float     = 1.      # north position
        self.east: float      = 2.      # east position
        self.altitude: float  = 3.      # altitude
        self.phi: float       = phi     # roll angle
        self.theta: float     = theta   # pitch angle
        self.psi: float       = psi     # yaw angle
pose = Pose()

# Initialize the points
p_3_i = np.array([[1],[2],[3]])
p_4_i = np.array([[3],[2],[1]])
p_5_b = np.array([[1],[2],[3]])
p_6_w = np.array([[1],[2],[3]])

# Calculate p_3^b
p_3_b = trans_i_to_b(pose, p_3_i)
print("p_3^b = ")
display(p_3_b)

# Calculate p_4_b
p_4_b = trans_i_to_b(pose, p_4_i)
print("p_4^b = ")
display(p_4_b)

# Calcualte p_5^i
p_5_i = trans_b_to_i(pose, p_5_b)
print("p_5^i")
display(p_5_i)

# Calculate p_6^i
p_6_b = np.linalg.inv(rot_b_to_s(alpha))@np.linalg.inv(rot_s_to_w(beta))@p_6_w
p_6_i = trans_b_to_i(pose, p_6_b)
print("p_6^i")
display(p_6_i)


p_3^b = 


matrix([[-1.77312124],
        [ 1.41812418],
        [ 5.55382435]])

p_4^b = 


matrix([[ 0.16896899],
        [-0.32143576],
        [ 4.45736789]])

p_5^i


matrix([[ 1.54090053],
        [ 4.2317526 ],
        [-0.04589997]])

p_6^i


matrix([[1.06667346],
        [3.91266605],
        [0.21516146]])

## Static analysis
Run the static code analysis (you must have zero static code analysis errors to get credit). You may not modify the static code analysis configuration files.

### ISORT
Run Isort:
```
python -m isort mav_sim book_assignments
```

Terminal output (should be nothing):

### MyPy
Run MyPy
```
python -m mypy mav_sim/chap2/ book_assignments
```

Terminal output (should indicate no error):
```
Success: no issues found in 15 source files
```

### Pylint
Run Pylint
```
python -m pylint --jobs 0 --rcfile .pylintrc mav_sim/chap2/ book_assignments/
```

Terminal output (should indicate `10/10`)
```
------------------------------------
Your code has been rated at 10.00/10
```

## Simple code checking
The following code does not need to change. It should just be used as a sanity check so that you know the code is implemented properly. The output should not have any lines reading `Failed test!`

In [1]:
from mav_sim.unit_tests.ch2_transforms_tests import run_all_tests
run_all_tests()

Starting rot_x test
End of test

Starting rot_y test
End of test

Starting rot_z test
End of test

Starting rot_v_to_v1 test
End of test

Starting rot_v1_to_v2 test
End of test

Starting rot_v2_to_b test
End of test

Starting rot_b_to_s test
End of test

Starting rot_s_to_w test
End of test

Starting rot_v_to_b test
End of test

Starting rot_b_to_v test
End of test

Starting trans_i_to_v test
End of test

Starting trans_v_to_i test
End of test

Starting trans_i_to_b test
End of test

Starting trans_b_to_i test
End of test

